<hr style="border-top:5px solid Gold; margin-top: 2px; margin-bottom: 0px">
<hr style="border-top:5px solid Silver; margin-top: 0px; margin-bottom: 0px">
<hr style="border-top:5px solid #DC9233; margin-top: 0px; margin-bottom: 0px">

# Medals at the Olympics

<hr style="border-top:5px solid Gold; margin-top: 2px; margin-bottom: 0px">
<hr style="border-top:5px solid Silver; margin-top: 0px; margin-bottom: 0px">
<hr style="border-top:5px solid #DC9233; margin-top: 0px; margin-bottom: 0px">

In [1]:
# Standard imports
import pandas as pd

# Visulation Imports
import matplotlib.pyplot as plt
import seaborn as sns

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Modeling imports
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import sklearn.preprocessing
import statsmodels.api as sm
from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# turn off pink boxes
import warnings
warnings.filterwarnings("ignore")

In [2]:
# reading in the csv and saving it to df
df = pd.read_csv('athlete_events.csv')